In [2]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [6]:
from transformers import BertModel, BertForMaskedLM, BertConfig,BertTokenizer, PreTrainedEncoderDecoder

# Initializing a BERT bert-base-uncased style configuration
configuration = BertConfig()

# Initializing a model from the bert-base-uncased style configuration
model = BertModel(configuration)

# Accessing the model configuration
configuration = model.config

In [69]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
input_ids = torch.tensor(tokenizer.encode("raise an exception of class ValueError, with string \"Key '%s' not found\" as argument, where '%s' is substituted for key.", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
# input_ids[0,7] = tokenizer.mask_token_id
# input_ids = input_ids[:,:-1]
# input_ids[0,9] = tokenizer.mask_token_id
print([tokenizer.convert_ids_to_tokens([predicted_index]) for predicted_index in input_ids[0]])
outputs = model(input_ids, masked_lm_labels=input_ids)
loss, prediction_scores = outputs[:2]

[['[CLS]'], ['raise'], ['an'], ['exception'], ['of'], ['class'], ['value'], ['##er'], ['##ror'], [','], ['with'], ['string'], ['"'], ['key'], ["'"], ['%'], ['s'], ["'"], ['not'], ['found'], ['"'], ['as'], ['argument'], [','], ['where'], ["'"], ['%'], ['s'], ["'"], ['is'], ['substituted'], ['for'], ['key'], ['.'], ['[SEP]']]


In [23]:
import json
json.dumps({None: None})

'{"null": null}'

In [21]:
import pprint as pp
model.state_dict().keys()

odict_keys(['encoder.embeddings.word_embeddings.weight', 'encoder.embeddings.position_embeddings.weight', 'encoder.embeddings.token_type_embeddings.weight', 'encoder.embeddings.LayerNorm.weight', 'encoder.embeddings.LayerNorm.bias', 'encoder.encoder.layer.0.attention.self.query.weight', 'encoder.encoder.layer.0.attention.self.query.bias', 'encoder.encoder.layer.0.attention.self.key.weight', 'encoder.encoder.layer.0.attention.self.key.bias', 'encoder.encoder.layer.0.attention.self.value.weight', 'encoder.encoder.layer.0.attention.self.value.bias', 'encoder.encoder.layer.0.attention.output.dense.weight', 'encoder.encoder.layer.0.attention.output.dense.bias', 'encoder.encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.encoder.layer.0.intermediate.dense.weight', 'encoder.encoder.layer.0.intermediate.dense.bias', 'encoder.encoder.layer.0.output.dense.weight', 'encoder.encoder.layer.0.output.dense.bias', 'encoder.encoder.la

In [67]:
print(prediction_scores.shape)
print()
max_ids = torch.argsort(prediction_scores[0,7], dim=0, descending=True)[:5]
print(max_ids)
[tokenizer.convert_ids_to_tokens([predicted_index]) for predicted_index in max_ids]

torch.Size([1, 8, 30522])

tensor([1000, 1012, 1007, 1996, 1006])


[['"'], ['.'], [')'], ['the'], ['(']]

In [52]:
print(prediction_scores.shape)
print()
max_ids = torch.argsort(prediction_scores[0,9], dim=0, descending=True)[:5]
print(max_ids)
[tokenizer.convert_ids_to_tokens([predicted_index]) for predicted_index in max_ids]

torch.Size([1, 12, 30522])

tensor([2665, 3730, 3585, 3652, 3376])


[['green'], ['soft'], ['golden'], ['growing'], ['beautiful']]

In [15]:
predicted_index = torch.argmax(prediction_scores[0], dim=1)[1].item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
predicted_token

'.'

In [9]:
prediction_scores.shape

torch.Size([1, 8, 30522])

In [10]:
input_ids

tensor([[  101,  7592,  1010,  2026,  3899,  2003, 10140,   102]])

# COMBINERS

In [2]:
import torch
from transformers import BertTokenizer, Model2Model, PreTrainedEncoderDecoder

# OPTIONAL: if you want to have more information on what's happening under the hood, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Encode the input to the encoder (the question)
question = "Who was Jim Henson?"
encoded_question = tokenizer.encode(question)

# Encode the input to the decoder (the answer)
answer = "Jim Henson was a puppeteer"
encoded_answer = tokenizer.encode(answer)

# Convert inputs to PyTorch tensors
question_tensor = torch.tensor([encoded_question])
answer_tensor = torch.tensor([encoded_answer])

To use data.metrics please install scikit-learn. See https://scikit-learn.org/stable/index.html
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [6]:
# In order to compute the loss we need to provide language model
# labels (the token ids that the model should have produced) to
# the decoder.
lm_labels =  encoded_answer
labels_tensor = torch.tensor([lm_labels])

# Load pre-trained model (weights)
model = Model2Model.from_pretrained('bert-base-uncased')

# Set the model in evaluation mode to deactivate the DropOut modules
# This is IMPORTANT to have reproducible results during evaluation!
model.eval()

# If you have a GPU, put everything on cuda
question_tensor = question_tensor.to('cuda')
answer_tensor = answer_tensor.to('cuda')
labels_tensor = labels_tensor.to('cuda')
model.to('cuda')

# Predict hidden states features for each layer
with torch.no_grad():
    # See the models docstrings for the detail of the inputs
    outputs = model(question_tensor, answer_tensor, decoder_lm_labels=labels_tensor)
    # Transformers models always output tuples.
    # See the models docstrings for the detail of all the outputs
    # In our case, the first element is the value of the LM loss 
    lm_loss = outputs[0]

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
INFO:transformers.configuration_utils:Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_siz

In [3]:
# Let's re-use the previous question
question = "Who was Jim Henson?"
encoded_question = tokenizer.encode(question)
question_tensor = torch.tensor([encoded_question])

# This time we try to generate the answer, so we start with an empty sequence
answer = "[CLS] He is [MASK]"
encoded_answer = tokenizer.encode(answer, add_special_tokens=False)
answer_tensor = torch.tensor([encoded_answer])

# Load pre-trained model (weights)
model = PreTrainedEncoderDecoder.from_pretrained('bert-base-uncased', 'bert-base-uncased')
model.eval()

# If you have a GPU, put everything on cuda
question_tensor = question_tensor.to('cuda')
answer_tensor = answer_tensor.to('cuda')
model.to('cuda')

# Predict all tokens
with torch.no_grad():
    outputs = model(question_tensor, answer_tensor)
    predictions = outputs[0]

# confirm we were able to predict 'jim'
predicted_index = torch.argmax(predictions[0, -1]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
predicted_token

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
INFO:transformers.configuration_utils:Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_siz

'.'